In [1]:
import pandas as pd
import  matplotlib

# Common  functions

In [2]:
def plot_default_per_colm_value(df, colmn_name):
    """
        plot the defaulting distribution per column categories
        Input:
            df: DataFrame of interest
            colmn_name: the column we are interested to visualize against default
        Output:
            defaulting distribution per column categories
    """
    defaulting_per_colmn = df.groupby('default')[colmn_name].value_counts().unstack(0)
    
    plot = defaulting_per_colmn.plot(kind="bar", figsize=(15, 5))
    plot.set_title("Distribution of defaulting per " + colmn_name)
    
    #print(defaulting_per_colmn)
    
    defaulting_per_colmn = defaulting_per_colmn.div(defaulting_per_colmn.sum(axis=1), axis=0)
    
    return defaulting_per_colmn

# Load the Data

In [3]:
data = pd.read_csv("../data/dataset.csv", sep=";")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99976 entries, 0 to 99975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 99976 non-null  object 
 1   default                              89976 non-null  float64
 2   account_amount_added_12_24m          99976 non-null  int64  
 3   account_days_in_dc_12_24m            88140 non-null  float64
 4   account_days_in_rem_12_24m           88140 non-null  float64
 5   account_days_in_term_12_24m          88140 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  40661 non-null  float64
 7   account_status                       45603 non-null  float64
 8   account_worst_status_0_3m            45603 non-null  float64
 9   account_worst_status_12_24m          33215 non-null  float64
 10  account_worst_status_3_6m            42274 non-null  float64
 11  account_worst_status_6_12m  

## Separate the data into training and testing: 
 - Testing data is the portion of data having default as NA

In [5]:
train_data = data[data['default'].notnull()]
train_data.to_csv('../data/training_dataset.csv', index=False)

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89976 entries, 0 to 89975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 89976 non-null  object 
 1   default                              89976 non-null  float64
 2   account_amount_added_12_24m          89976 non-null  int64  
 3   account_days_in_dc_12_24m            79293 non-null  float64
 4   account_days_in_rem_12_24m           79293 non-null  float64
 5   account_days_in_term_12_24m          79293 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  36619 non-null  float64
 7   account_status                       41042 non-null  float64
 8   account_worst_status_0_3m            41042 non-null  float64
 9   account_worst_status_12_24m          29921 non-null  float64
 10  account_worst_status_3_6m            38038 non-null  float64
 11  account_worst_status_6_12m  

In [8]:
test_data = data[data['default'].isnull()]
test_data.to_csv('../data/testing_dataset.csv', index=False)

# EDA
 1. Visualize the label (default) and check for imbalancing
 2. Get better understanding on how user information and taste might lead to defaulting
 3. Understand attributes describing loan status
 4. Do in depth analysis of the account's statues
 5. Test how the increase of paid/added amounts might reduce the probability of defaulting
 6. Tap into who active invoices and dept history might correlate with defaulting
 7. Analyze the Written Off loans
 8. Determine if other attributes like time_hours is useful for our modeling

## 1. Visualize the label (default) and check for imbalancing
 - Data is very imbalanced: we might need to do over or under sampling
 - Defaulting occured only in 1.4% of the data

In [ ]:
train_data['default'].value_counts().plot(kind='pie')

In [ ]:
default_value_counts = train_data['default'].value_counts()
100 * default_value_counts[1.0]/ sum(default_value_counts)

## 2. Get better understanding on how user information and taste might lead to defaulting
 - All four attributes of interest (age, name_in_email, merchant_group, and merchant_category) have no null values

In [ ]:
min_age = train_data['age'].min()
max_age = train_data['age'].max()
train_data.groupby('default')['age'].hist(bins=[min_age, 25, 40, 60, max_age])

In [ ]:
plot_default_per_colm_value(train_data, 'name_in_email')

In [ ]:
plot_default_per_colm_value(train_data, 'merchant_group')

In [ ]:
default_per_permerchant_category = plot_default_per_colm_value(train_data, 'merchant_category')

In [ ]:
default_per_permerchant_category

### Conclusion
- For age:
    - The data was splitted based on generations (Generation Z, Millennials, Generation X, and older generations)
    - As expected millennials are more likely to be using the service
    - After visualizing the data, the hypothesis that younger generation will high tendency for defaulting than older generation is rejected
- For name_in_email:
     - No correlation between the name_in_email and defaulting were detected
- For merchant_group:
     - Each group have defaulting probability close to the general defaulting distribution
- For merchant_category:
     - Some categories have very high probability of defaulting ( compared to the general defaulting distribution)(e.g. Plants & Flowers, Sex toys, Tobacco)
     - Other categories like Bags & Wallets, Cleaning & Sanitary, and Safety products have norecords of defaulting

## 3. Understand attributes describing loan status
-  remittance (account_days_in_rem_12_24m, num_arch_rem_0_12m)
-  direct credit (account_days_in_dc_12_24m, num_arch_dc_0_12m, num_arch_dc_12_24m)
-  term loan (account_days_in_term_12_24m)
-  ok transactions ( num_arch_ok_0_12m, num_arch_ok_12_24m)

In [ ]:
days_colms = ['account_days_in_dc_12_24m', 'account_days_in_rem_12_24m', 'account_days_in_term_12_24m', 'num_arch_dc_0_12m', 'num_arch_dc_12_24m', 'num_arch_ok_0_12m', 'num_arch_ok_12_24m', 'num_arch_rem_0_12m']
train_data.boxplot(column=days_colms, figsize=(20, 5))  

In [ ]:
train_data_days = train_data[['default'] + days_colms].copy()

In [ ]:
train_data_days.describe(percentiles=[0.25, .5, .75, .85, .9, .95])

In [ ]:
for colmn in days_colms:
    train_data_days[colmn] = train_data_days[colmn].apply(lambda x: 0 if x==0  else(1 if x > 0 else "unavailable"))
    plot_default_per_colm_value(train_data_days, colmn)

### Conclusion 
- All eight attributes are mostly always Zero
- Regardless of the value assigned to any of the attributes the defaulting probability almost the same
- In data engineering phase, all null values will be replaced by Zero

## 4. Do in depth analysis of the account's statues, attributes of interest are
 - account_status 
 - account_worst_status_0_3m
 - account_worst_status_12_24m
 - account_worst_status_3_6m
 - account_worst_status_6_12m
 - worst_status_active_inv
 - status_last_archived_0_24m
 - status_2nd_last_archived_0_24m
 - status_3rd_last_archived_0_24m
 - status_max_archived_0_6_months
 - status_max_archived_0_12_months
 - status_max_archived_0_24_months

In [ ]:
status_colms = ['account_status', 'account_worst_status_0_3m', 'account_worst_status_12_24m', 'account_worst_status_3_6m', 'account_worst_status_6_12m', 'worst_status_active_inv', 'status_last_archived_0_24m', 'status_2nd_last_archived_0_24m', 'status_3rd_last_archived_0_24m', 'status_max_archived_0_6_months', 'status_max_archived_0_12_months', 'status_max_archived_0_24_months']
train_data.boxplot(column=status_colms, figsize=(15, 5),rot=90)  

In [ ]:
train_data_status = train_data[['default'] + status_colms].copy()
vc_nulls = train_data[status_colms].apply(lambda x: x.isnull().value_counts()).T[True]
vc_nulls

In [ ]:
train_data_status = train_data[['default'] + status_colms].copy()
train_data_status['max'] = train_data_status[status_colms].max(axis=1)
for colmn in (status_colms + ['max']):
    train_data_status[colmn] = train_data_status[colmn].apply(lambda x: x if x >= 0  else "unavailable")
    default_per_colname = plot_default_per_colm_value(train_data_status, colmn)
    #print(default_per_colname)

### Conclusion 
- All twelve attributes are categorical 
- Archived status:
    - can take one of the following values {0, 1, 2, 3, 4, 5}; zero means good status and Five means very bad status
    - Have no null values
- Active status:
    - From my understanding, account_status is kind of a weighted average of the prvious account status
    - can take one of the following values {1, 2, 3, 4}; no Zero  nor Five!
    - For each status more than 50% of the values are missing
- Regardless of the value assigned to any of the attributes the defaulting probability almost the same
- In data engineering phase:
    - new attribute will be created capturing the maximum statues for active loans.
    - Test if using only three attributes, namely (account_status, status_max_archived_0_24_months, maximum statues for active loans) will give good results as all other attributes are kind of correlated to those three attributes
    - Null values will not be filled nor the record will be deleted, rather I will use algorithms that support missing values 

## 5. Test how the increase of paid/added amounts might reduce the probability of defaulting
- account_amount_added_12_24m
- max_paid_inv_0_12m
- max_paid_inv_0_24m
- sum_capital_paid_account_0_12m
- sum_capital_paid_account_12_24m

In [ ]:
payment_colms = ['account_amount_added_12_24m', 'max_paid_inv_0_12m', 'max_paid_inv_0_24m', 'sum_capital_paid_account_0_12m', 'sum_capital_paid_account_12_24m']
train_data.boxplot(column=payment_colms, figsize=(20, 5))  

In [ ]:
for colmn in payment_colms:
    train_data.plot.scatter(x='default', y=colmn, c='DarkBlue')

In [ ]:
# calc sum_capital_paid_account_0_24m
temp_df = train_data[['default'] + payment_colms]
temp_df['sum_capital_paid_account_0_24m'] = temp_df['sum_capital_paid_account_0_12m'] + temp_df['sum_capital_paid_account_12_24m']
temp_df.plot.scatter(x='default', y='sum_capital_paid_account_0_24m', c='DarkBlue')

### Conclusion 
- Increase in ammounts added or paid lower the probability of defaulting
- In modeling phase, only max_paid_inv_0_24m, account_amount_added_12_24m, and sum_capital_paid_account_0_24m will be used

## 6. Tap into who active invoices and dept history might correlate with defaulting
 - account_incoming_debt_vs_paid_0_24m
 - num_active_div_by_paid_inv_0_12m
 - num_active_inv
 - num_unpaid_bills
 - recovery_debt

In [ ]:
dept_colms = ['account_incoming_debt_vs_paid_0_24m', 'num_active_div_by_paid_inv_0_12m', 'num_active_inv', 'num_unpaid_bills', 'recovery_debt']

train_data.boxplot(column=dept_colms, figsize=(20, 5))  

In [ ]:
dept_train_data = train_data[['default'] + dept_colms]
dept_train_data['num_of_paid_inv_0_12m'] = dept_train_data.apply(lambda row: row['num_active_inv'] / row['num_active_div_by_paid_inv_0_12m'], axis=1)
dept_colms += ['num_of_paid_inv_0_12m']
dept_train_data.describe(percentiles=[0.25, .5, .75, .85, .9, .95])

In [ ]:
dept_train_data = dept_train_data.fillna(0)
#dept_train_data['num_of_paid_inv_0_12m'] = dept_train_data['num_of_paid_inv_0_12m'].fillna(0)
for colmn in dept_colms:
    dept_train_data.plot.scatter(x='default', y=colmn, c='DarkBlue')

### Conclusion 
- 63% of account_incoming_debt_vs_paid_0_24m are missing so it will be deleted
- more than 95% of the data in recovery_debt is Zero and all non zero rows, no much information is obtained (will not be used in modelling)
- Both num_active_inv & num_unpaid_bills kind of having same graph with different scaling. Going forward only num_unpaid_bills will be kept
- In data engineering phase, the  num_of_paid_inv_0_12m will be calculated
- In modeling phase, only num_active_div_by_paid_inv_0_12m, num_unpaid_bills, and num_of_paid_inv_0_12m will be used

## 7. Analyze the Written Off loans
 - num_arch_written_off_0_12m
 - num_arch_written_off_12_24m

In [ ]:
arch_colms = [ 'num_arch_written_off_0_12m', 'num_arch_written_off_12_24m']
train_data.boxplot(column=arch_colms, figsize=(15, 5))  

In [ ]:
train_data_arch = train_data[['default'] + arch_colms].copy()
vc_nulls = train_data[arch_colms].apply(lambda x: x.isnull().value_counts()).T[True]
vc_nulls

In [ ]:
train_data_arch = train_data[['default'] + arch_colms].copy()
for colmn in arch_colms:
    default_per_colname = plot_default_per_colm_value(train_data_arch, colmn)
    print(default_per_colname)

### Conclusion 
- Both attributes are mostly always Zero
- Regardless of the value assigned to any of the attributes the defaulting probability almost the same
- In data engineering phase, both columns will be removed

## 8. Determine if other attributes like age is useful for our modeling
- avg_payment_span_0_12m
- avg_payment_span_0_3m
- time_hours

In [ ]:
train_data.boxplot(column=['time_hours','avg_payment_span_0_12m','avg_payment_span_0_3m'], figsize=(20, 5))  

In [ ]:
train_data[['time_hours','avg_payment_span_0_12m','avg_payment_span_0_3m']].describe(percentiles=[0.25, .5, .75, .85, .9, .95])

In [ ]:
train_data.groupby('default')['time_hours'].hist()

In [ ]:
train_data.groupby('default')['has_paid'].value_counts()

### Conclusion 
- Both avg_payment_span_0_12m & avg_payment_span_0_3m are mostly missing (Will be deleted)
- Given the short span of time_hours, it is hard to infer what it represents exactly

# Final Conclusion
- **Train-Test split:**
 - 10% of the data had missing 'default' values, and will be used as the testing data
 - The rest 90% of the data will be used for modelling (training data) 
 
 <br/>
 <br/>
 
- **Label distribution is skewed:** only 1.4% of the training data is labled as default

<br/>
 <br/>

- **New features that will be added:**
  - sum_capital_paid_account_0_24m: sum of both sum_capital_paid_account_0_12m and sum_capital_paid_account_12_24m
  - num_of_paid_inv_0_12m: calculated as num_active_inv /  num_active_div_by_paid_inv_0_12m
  - status_max_active_0_24: the maximum statues for active loans (account_worst_status_0_3m, account_worst_status_12_24m, account_worst_status_3_6m, account_worst_status_6_12m)
  
  <br/>
 <br/>
  
  
- **Attributes which will not be used for modeling:**
 - age: the hypothesis that younger generation will high tendency for defaulting than older generation is rejected, as it turns out all gens have almost same probability of defaulting
 - name_in_email: No correlation between the name_in_email and defaulting were detected
 - For merchant_group: Each group have defaulting probability close to the general defaulting distribution
 - time_hours: it is hard to infer what it represents exactly, and dosent seem to have correlation with defaulting
 - Attributes with more than 50% of missing values:
   1. avg_payment_span_0_12m (76%)
   2. avg_payment_span_0_3m (51%)
   3. account_incoming_debt_vs_paid_0_24m (63%)
 - Attributes which more than 97% or so of the times have the same value:
   1. num_arch_written_off_0_12m (Zero)
   2. num_arch_written_off_12_24m (Zero)
   3. recovery_debt (Zero)
   4. account_days_in_dc_12_24m (Zero)
   5. account_days_in_term_12_24m (Zero)
   6. num_arch_dc_0_12m (Zero)
   7. num_arch_dc_12_24m (Zero)
 - Attributes having high correlation with others:
   1. num_active_inv: num_active_inv & num_unpaid_bills kind of having same graph with different scaling.
   2. max_paid_inv_0_12m: correlated to max_paid_inv_0_24m
   3. sum_capital_paid_account_0_12m: correlated with sum_capital_paid_account_0_24m
   4. sum_capital_paid_account_12_24m: correlated with sum_capital_paid_account_0_24m
   5. account_worst_status_0_3m: correlated with status_max_active_0_24
   6. account_worst_status_12_24m: correlated with status_max_active_0_24
   7. account_worst_status_3_6m: correlated with status_max_active_0_24
   8. account_worst_status_6_12m: correlated with status_max_active_0_24